###  날씨 계산식

DPI는 날씨 요소(강수량, 대기질, 불쾌지수 등)를 점수로 변환해 데이트 적합성을 평가하는 지표입니다. 최대 100점으로, 점수가 높을수록 데이트하기 좋은 날씨를 의미합니다. 주요 요소는:

강수량 (precipitation): 비, 눈 등이 실외 활동에 큰 영향을 미침.
대기질 (AQI, air quality index): 미세먼지가 실외에서 더 문제됨.
불쾌지수 (discomfort index): 온도와 습도가 실내외 모두에 영향을 미침.
기타: 주말/공휴일 보너스 등.
실내 DPI와 실외 DPI의 차이
실내와 실외 데이트는 날씨 조건에 대한 민감도가 다르기 때문에, DPI 계산 시 가중치를 조정합니다.

1. 실외 DPI (Outdoor DPI)
설명: 공원 산책, 야외 카페 등 실외 활동을 위한 날씨 적합도를 평가.
특징:
강수량과 대기질에 민감: 비가 오거나 미세먼지가 심하면 실외 활동이 어려워 점수가 크게 낮아짐.
가중치: 강수량(rain_score)과 대기질(aqi_score) 점수를 100% 반영.
계산 예시:
강수량 0mm → rain_score = 30
AQI 50 → aqi_score = 20
불쾌지수 60 → discomfort_score = 25
실외 DPI = 30 + 20 + 25 = 75
2. 실내 DPI (Indoor DPI)
설명: 카페, 영화관 등 실내 활동을 위한 날씨 적합도를 평가.
특징:
강수량과 대기질 영향 감소: 실내에서는 비나 미세먼지가 덜 중요하므로 가중치를 낮춤(0.5배).
불쾌지수 중심: 온도와 습도가 실내 환경에서 더 큰 영향을 미침.
계산 예시:
강수량 0mm → rain_score = 30 * 0.5 = 15
AQI 50 → aqi_score = 20 * 0.5 = 10
불쾌지수 60 → discomfort_score = 25
실내 DPI = 25 + 15 + 10 = 50
3. 공통 점수 조정
실외 활동 불가 조건: 강수량 10mm 초과, AQI 200 초과, 불쾌지수 80 이상 또는 40 이하 → DPI = 0 (실내/실외 모두 적용).
주말/공휴일 보너스: 주말이면 +5점 (실내/실외 모두 적용).
코드에서 확인
이전 코드의 calculate_dpi 함수에서 실내/실외 DPI 계산 로직을 확인할 수 있습니다:

python

접기

자동 줄바꿈

복사
if outdoor:
    dpi = rain_score + aqi_score + discomfort_score  # 실외: 모든 요소 100% 반영
else:
    dpi = discomfort_score + (rain_score * 0.5) + (aqi_score * 0.5)  # 실내: 강수와 AQI 50% 반영
실제 예시 (DB 데이터 기반)
DB에서 가져온 데이터로 계산한 예시:

데이터: 강남 MICE 관광특구, temp=10.1, precipitation=1, air_idx=127.0, sky_stts=흐림, humidity=55, wind_spd=2.0, datetime=2025-03-27 21:00 (목요일)
계산:
sunny = 0 (흐림)
rainy = 1 (비)
rainfall = 1 → rain_score = 15
aqi = 127 → aqi_score = 8
discomfort_index ≈ 51 → discomfort_score = 20
실외 DPI = 15 + 8 + 20 = 43
실내 DPI = 20 + (15 * 0.5) + (8 * 0.5) = 20 + 7.5 + 4 = 31.5
차이점 요약
요소	실외 DPI	실내 DPI
강수량	100% 반영	50% 반영
대기질	100% 반영	50% 반영
불쾌지수	100% 반영	100% 반영
적용 예	야외 산책	카페 데이트
실외 DPI가 높으면 날씨가 맑고 공기가 좋아 야외 활동에 적합.
실내 DPI가 높으면 날씨가 덜 중요하고 실내에서 쾌적한 조건.

In [ ]:
import math

# DPI 계산 함수
def calculate_dpi(sunny, rainy, temperature, temD, wind, weekend, holiday, aqi, outdoor=True):
    """
    Parameters:
    - sunny: 맑거나 부분맑음이면 1, 아니면 0
    - rainy: 비/눈/폭풍이면 1, 아니면 0
    - temperature: 최고 기온 (°C)
    - temD: 일교차 (최고기온 - 최저기온, °C)
    - wind: 풍속 (m/s)
    - weekend: 주말이면 1, 아니면 0
    - holiday: 공휴일이면 1, 아니면 0
    - aqi: 통합대기환경지수 (0-1 사이 값, 실제로는 0-500 범위를 0-1로 변환)
    - outdoor: 실외 데이트면 True, 실내면 False
    """
    
    # 강수량 계산 (rainy 기반으로 가정, 실제 mm 값은 입력 필요)
    rainfall = 15 if rainy == 1 else 0  # 비가 오면 15mm로 가정, 실제 값으로 대체 가능
    
    # 불쾌지수 계산 (온도와 상대습도 필요, 여기서는 단순화)
    # 상대습도(humidity)는 고정값 50%로 가정, 실제 데이터로 대체 가능
    humidity = 50  # 상대습도 (%)
    discomfort_index = 0.81 * temperature + 0.01 * humidity * (0.99 * temperature - 14.3) + 46.3
    
    # 1. 강수 요소 점수
    if rainfall == 0:
        rain_score = 30  # 최적
    elif 0 < rainfall <= 5:
        rain_score = 15  # 약간 불편
    elif 5 < rainfall <= 10:
        rain_score = 5   # 실외 활동 어려움
    else:
        rain_score = 0   # 실외 활동 불가
    
    # 2. 미세먼지 요소 점수 (AQI는 0-1을 0-500으로 변환 가정)
    aqi_value = aqi * 500  # 0-1 값을 0-500으로 스케일링
    if aqi_value <= 50:
        aqi_score = 20  # 좋음
    elif aqi_value <= 100:
        aqi_score = 15  # 보통
    elif aqi_value <= 150:
        aqi_score = 8   # 약간 나쁨
    elif aqi_value <= 200:
        aqi_score = 3   # 나쁨
    else:
        aqi_score = 0   # 실외 활동 불가
    
    # 3. 불쾌지수 요소 점수
    if 55 <= discomfort_index <= 65:
        discomfort_score = 25  # 쾌적
    elif (65 < discomfort_index <= 70) or (50 <= discomfort_index < 55):
        discomfort_score = 20  # 약간 덥거나 춥지만 괜찮음
    elif (70 < discomfort_index <= 75) or (45 <= discomfort_index < 50):
        discomfort_score = 15  # 불쾌감 시작
    elif (75 < discomfort_index <= 80) or (40 <= discomfort_index < 45):
        discomfort_score = 10  # 불편
    else:
        discomfort_score = 5   # 실외 활동 불가
    
    # 4. 실외 활동 불가 조건 체크
    if rainfall > 10 or aqi_value > 200 or discomfort_index >= 80 or discomfort_index <= 40:
        dpi = 0  # 실외 활동 불가
    else:
        # 기본 DPI 계산
        dpi = rain_score + aqi_score + discomfort_score
        
        # 실내/실외 데이트 유형별 가중치 조정
        if outdoor:
            # 실외 데이트: 강수와 미세먼지 가중치 높임 (각각 1.0 유지)
            dpi = dpi
        else:
            # 실내 데이트: 강수와 미세먼지 가중치 낮춤 (0.5배 적용)
            dpi = discomfort_score + (rain_score * 0.5) + (aqi_score * 0.5)
    
    # 주말 및 공휴일 보너스 (선택적)
    if weekend or holiday:
        dpi += 5  # 주말/공휴일이면 약간의 보너스 점수
    
    # 최대 점수 제한 (100점 기준으로 조정 가능)
    dpi = min(dpi, 100)
    
    return dpi

# 테스트 코드
if __name__ == "__main__":
    # 예시 값 (한글 주석으로 넣을 값 표시)
    sunny = 1      # 맑거나 부분맑음이면 1, 아니면 0
    rainy = 0      # 비/눈/폭풍이면 1, 아니면 0
    temperature = 25  # 최고 기온 (°C)
    temD = 10      # 일교차 (°C)
    wind = 2.5     # 풍속 (m/s)
    weekend = 1    # 주말이면 1, 아니면 0
    holiday = 0    # 공휴일이면 1, 아니면 0
    aqi = 0.1      # 통합대기환경지수 (0-1 사이, 예: 0.1은 AQI 50에 해당)
    
    # 실외 데이트 DPI
    dpi_outdoor = calculate_dpi(sunny, rainy, temperature, temD, wind, weekend, holiday, aqi, outdoor=True)
    print(f"실외 데이트 DPI: {dpi_outdoor}")
    
    # 실내 데이트 DPI
    dpi_indoor = calculate_dpi(sunny, rainy, temperature, temD, wind, weekend, holiday, aqi, outdoor=False)
    print(f"실내 데이트 DPI: {dpi_indoor}")

코드 설명

강수 요소 (rain_score):
rainy 값으로 강수량을 가정(15mm 또는 0mm). 실제 강수량 데이터를 넣으려면 rainfall 변수를 수정하세요.

미세먼지 요소 (aqi_score):
aqi는 0-1 사이 값으로 입력받아 0-500으로 변환 후 점수화.

불쾌지수 (discomfort_score):
온도와 상대습도를 사용해 계산. 현재는 습도를 50%로 고정했지만, 실제 데이터로 대체 가능.

실외 활동 불가 조건:
강수량 10mm 초과, AQI 200 초과, 불쾌지수 80 이상/40 이하 시 DPI를 0으로 설정.

데이트 유형별 가중치:
outdoor=True: 실외 데이트는 강수와 미세먼지 점수를 그대로 사용.
outdoor=False: 실내 데이트는 강수와 미세먼지 점수를 0.5배로 줄임.
보너스 점수:
주말(weekend) 또는 공휴일(holiday)이면 +5점 추가.
실행 예시
입력: sunny=1, rainy=0, temperature=25, temD=10, wind=2.5, weekend=1, holiday=0, aqi=0.1
불쾌지수 ≈ 68 (20점)
강수: 30점
AQI: 20점 (50 이하)
실외 DPI: 30 + 20 + 20 + 5 (주말 보너스) = 75
실내 DPI: 20 + (30 * 0.5) + (20 * 0.5) + 5 = 50
출력:
text

접기

자동 줄바꿈

복사
실외 데이트 DPI: 75
실내 데이트 DPI: 50
주석으로 넣을 값
sunny: # 맑거나 부분맑음이면 1, 아니면 0
rainy: # 비/눈/폭풍이면 1, 아니면 0
temperature: # 최고 기온 (°C)
temD: # 일교차 (°C)
wind: # 풍속 (m/s)
weekend: # 주말이면 1, 아니면 0
holiday: # 공휴일이면 1, 아니면 0
aqi: # 통합대기환경지수 (0-1 사이)
추가 개선 제안
실제 강수량: rainfall에 API로 가져온 mm 값을 넣으면 더 정확함.
습도: 불쾌지수 계산에 실제 습도 데이터를 추가 가능.
풍속: wind를 점수화하려면 별도 기준 추가 가능(예: 10m/s 이상이면 감점).

### db에서 가져와서 날씨 계산하는 코드

In [4]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# 네 샘플 데이터를 DataFrame으로 변환
data = [
    (None, '4·19 카페거리', '수유역', '강북구', 900, 1000, 7, 0, 20, '없음', '82.0', '흐림', 68, 1.0, 82),
    # ... (108개 행 전체 데이터 사용)
]
columns = ['datetime', 'region', 'station', 'gu', 'min_population', 'max_population', 'temp', 
           'precipitation', 'rain_chance', 'precpt_type', 'air_idx', 'sky_stts', 'humidity', 'wind_spd', 'uv_idx']
df = pd.DataFrame(data, columns=columns)

# DPI 계산 함수
def calculate_dpi(row, outdoor=True):
    sunny = 1 if row['sky_stts'] == '맑음' else 0
    rainy = 1 if row['precpt_type'] in ['비', '눈'] else 0
    temperature = row['temp']
    rainfall = row['precipitation']
    aqi = float(row['air_idx']) if row['air_idx'] != '점검중' and row['air_idx'] is not None else 100
    humidity = row['humidity']
    # datetime이 None이므로 주말 가정 제거
    weekend = 0
    
    discomfort_index = 0.81 * temperature + 0.01 * humidity * (0.99 * temperature - 14.3) + 46.3
    
    rain_score = 30 if rainfall == 0 else 15 if rainfall <= 5 else 5 if rainfall <= 10 else 0
    aqi_score = 20 if aqi <= 50 else 15 if aqi <= 100 else 8 if aqi <= 150 else 3 if aqi <= 200 else 0
    discomfort_score = (25 if 55 <= discomfort_index <= 65 else 
                       20 if (65 < discomfort_index <= 70 or 50 <= discomfort_index < 55) else 
                       15 if (70 < discomfort_index <= 75 or 45 <= discomfort_index < 50) else 
                       10 if (75 < discomfort_index <= 80 or 40 <= discomfort_index < 45) else 5)
    
    if rainfall > 10 or aqi > 200 or discomfort_index >= 80 or discomfort_index <= 40:
        dpi = 0
    else:
        dpi = rain_score + aqi_score + discomfort_score if outdoor else discomfort_score + (rain_score * 0.5) + (aqi_score * 0.5)
    
    if weekend:
        dpi += 5
    return min(dpi, 100)

# DPI 계산 적용
df['dpi_outdoor'] = df.apply(lambda row: calculate_dpi(row, outdoor=True), axis=1)
df['dpi_indoor'] = df.apply(lambda row: calculate_dpi(row, outdoor=False), axis=1)

# 결과 확인
print(df[['region', 'station', 'dpi_outdoor', 'dpi_indoor']].head())

      region station  dpi_outdoor  dpi_indoor
0  4·19 카페거리     수유역           60        37.5
